In [ ]:
import numpy as np
import requests
import base64

class QuantumRandomGenerator:
    def __init__(self, api_token: str):
        self.api_token = api_token
        self.base_url = 'https://api-qxeaas.quantumemotion.com/entropy'
        self.headers = {
            'Authorization': f'Bearer {self.api_token}'
        }

    def get_quantum_random(self, num_bytes: int) -> np.ndarray:
        """
        Fetch quantum random bytes from the API.

        Args:
            num_bytes (int): Number of random bytes to request (max 512).

        Returns:
            np.ndarray: Array of random bytes.
        """
        # if num_bytes > 512:
        #     raise ValueError("num_bytes cannot exceed 512.")
        # try:
        #     response = requests.get(
        #         self.base_url,
        #         headers=self.headers,
        #         params={'size': num_bytes}
        #     )
        #     response.raise_for_status()
        #     data = response.json()
        #     qrng_base64 = data['random_number']
        #     qrng_bytes = base64.b64decode(qrng_base64)
        #     qrng_numbers = np.frombuffer(qrng_bytes, dtype=np.uint8)
        #     return qrng_numbers
        # except requests.exceptions.RequestException as e:
        #     raise Exception(f"Error fetching quantum random numbers: {str(e)}")

        if num_bytes > 512:
            raise ValueError("num_bytes cannot exceed 512.")

        # Simulated API response for quantum random number generation
        qrng_bytes = np.random.bytes(num_bytes)  # Generate random bytes locally
        qrng_base64 = base64.b64encode(qrng_bytes).decode('utf-8')  # Encode in base64

        # Decode as if received from the API
        qrng_decoded_bytes = base64.b64decode(qrng_base64)
        qrng_numbers = np.frombuffer(qrng_decoded_bytes, dtype=np.uint8)

        return qrng_numbers


In [ ]:
import torch
import torch.nn as nn
import os

def quantum_random_uniform(shape, qrng: QuantumRandomGenerator, a=0.0, b=1.0):
    num_elements = np.prod(shape)
    num_bytes_needed = num_elements * 4  # 4 bytes per float32
    # Fetch quantum random bytes in chunks of up to 512 bytes
    random_bytes = bytearray()
    while len(random_bytes) < num_bytes_needed:
        bytes_to_fetch = min(256, num_bytes_needed - len(random_bytes))
        qrng_bytes = qrng.get_quantum_random(bytes_to_fetch)
        random_bytes.extend(qrng_bytes)
    # Convert bytes to float32 numbers
    total_bytes = bytes(random_bytes[:num_bytes_needed])
    random_uint32 = np.frombuffer(total_bytes, dtype=np.uint32)
    random_floats = random_uint32.astype(np.float32)
    random_floats /= np.iinfo(np.uint32).max  # Normalize to [0,1)
    random_floats = random_floats * (b - a) + a  # Scale to [a,b)
    return torch.from_numpy(random_floats).view(shape)

class QuantumInitializedLinear(nn.Linear):
    def reset_parameters(self):
        api_token = os.getenv('API_TOKEN')
        qrng = QuantumRandomGenerator(api_token=api_token)
        self.weight.data = quantum_random_uniform(self.weight.shape, qrng, a=-0.1, b=0.1)
        if self.bias is not None:
            self.bias.data = quantum_random_uniform(self.bias.shape, qrng, a=-0.1, b=0.1)


In [ ]:
import torchvision
import torchvision.transforms as transforms

# Load dataset
transform = transforms.Compose([transforms.ToTensor()])
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)

# Initialize QRNG
api_token = os.getenv('API_TOKEN')
qrng = QuantumRandomGenerator(api_token=api_token)

# Define model
class QuantumModel(nn.Module):
	def __init__(self):
		super(QuantumModel, self).__init__()
		self.fc1 = QuantumInitializedLinear(28*28, 128)
		self.fc2 = QuantumInitializedLinear(128, 64)
		self.fc3 = QuantumInitializedLinear(64, 10)

	def forward(self, x):
		x = x.view(-1, 28*28)
		x = F.relu(self.fc1(x))
		x = quantum_dropout(x, p=0.5, training=self.training, qrng=qrng)
		x = F.relu(self.fc2(x))
		x = quantum_dropout(x, p=0.5, training=self.training, qrng=qrng)
		x = self.fc3(x)
		return F.log_softmax(x, dim=1)

# Instantiate model
model = QuantumModel()

# Define optimizer and loss function
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = nn.NLLLoss()


In [ ]:
def train_model(model, train_loader, optimizer, criterion, epochs=10):
    model.train()
    for epoch in range(1, epochs + 1):
        epoch_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        num_batches = len(train_loader)

        print(f"\nEpoch {epoch}/{epochs}")
        print("-" * 30)

        for batch_idx, (data, target) in enumerate(train_loader, 1):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            batch_loss = loss.item()
            epoch_loss += batch_loss

            # Calculate accuracy for this batch
            _, predicted = torch.max(output, 1)
            correct_predictions += (predicted == target).sum().item()
            total_samples += target.size(0)

            # Print batch-level information
            if batch_idx % 100 == 0 or batch_idx == num_batches:  # Update every 100 batches or at end of epoch
                print(f"Batch {batch_idx}/{num_batches} - Loss: {batch_loss:.4f}")

        # Compute and print epoch average loss
        avg_epoch_loss = epoch_loss / num_batches
        accuracy = 100 * correct_predictions / total_samples
        print(f"End of Epoch {epoch} - Average Loss: {avg_epoch_loss:.4f} - Accuracy: {accuracy:.2f}%")

    print("\nTraining complete.")
    return model

# Train model
trained_model = train_model(model, train_loader, optimizer, criterion)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Load dataset
transform = transforms.Compose([transforms.ToTensor()])
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)

# Define classical model with standard weight initialization and dropout
class ClassicalModel(nn.Module):
    def __init__(self):
        super(ClassicalModel, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

# Define classical training function with accuracy tracking
def train_classical_model(model, train_loader, optimizer, criterion, epochs=10):
    model.train()
    for epoch in range(1, epochs + 1):
        epoch_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        num_batches = len(train_loader)

        print(f"\nEpoch {epoch}/{epochs}")
        print("-" * 30)

        for batch_idx, (data, target) in enumerate(train_loader, 1):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            batch_loss = loss.item()
            epoch_loss += batch_loss

            # Calculate accuracy for this batch
            _, predicted = torch.max(output, 1)
            correct_predictions += (predicted == target).sum().item()
            total_samples += target.size(0)

            # Print batch-level information
            if batch_idx % 100 == 0 or batch_idx == num_batches:
                print(f"Batch {batch_idx}/{num_batches} - Loss: {batch_loss:.4f}")

        # Compute and print epoch average loss
        avg_epoch_loss = epoch_loss / num_batches
        accuracy = 100 * correct_predictions / total_samples
        print(f"End of Epoch {epoch} - Average Loss: {avg_epoch_loss:.4f} - Accuracy: {accuracy:.2f}%")

    print("\nTraining complete.")
    return model

# Instantiate classical model
classical_model = ClassicalModel()

# Define optimizer and loss function for classical model
optimizer = optim.SGD(classical_model.parameters(), lr=0.01)
criterion = nn.NLLLoss()

# Train classical model
trained_classical_model = train_classical_model(classical_model, train_loader, optimizer, criterion)
